In [23]:
#The purpose of this notebook is to compute the basin stability of a network governed by the coupled oscillator model
#Import necessary packages
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import scipy.integrate
import scipy.stats
from scipy.linalg import circulant
from scipy.optimize import curve_fit
import time
import networkx as nx



In [150]:
#Define functions


def create_ensembles( pos, first_entries, second_entries, T, theta_s, omega_s):
    phase_group = []
    freq_group = []
    # For each index, set the `pos` element as the first entry, and others as `theta_s`
    for pval in first_entries:
        phase_tuple_ = [theta_s] * T  # Initialize a tuple with all elements set to theta_s
        phase_tuple_[pos] = pval  # Set the desired position with the first entry
        phase_group.append(tuple(phase_tuple_))  # Convert list to tuple

    for fval in second_entries:
        freq_tuple_ = [omega_s] * T  # Initialize a tuple with all elements set to theta_s
        freq_tuple_[pos] = fval  # Set the desired position with the first entry
        freq_group.append(tuple(freq_tuple_))  # Convert list to tuple
    return phase_group, freq_group



def kuramoto_function(phase,t,K,N,omega):
    phase_matr = np.tile(phase,(N,1))
    temp = np.multiply(K,np.sin(phase_matr-phase_matr.T))
    incr = omega + np.sum(temp,axis=1)
    return (incr)



def simulate_kuramoto(kuramoto, initial_θ, initial_ω, t):
    sol =scipy.integrate.odeint(kuramoto,y0=initial_θ,t=t,args=(K_ij,N,initial_ω),hmax=0.1,full_output=0,rtol=1e-4,atol=1e-4)
    ode_sol = kuramoto_function(sol[-1],0,K_ij,N,initial_ω)
    return sol, ode_sol
    


In [151]:
#### Model Parameters #### 
##########################
T= 3 # number of ensembles you want to test 
N= 3 #number of nodes
θ_s = np.arcsin(1) #synchronous phase state is arcsin(P/K)
ω_s = 0 # synchronous frequency


# Define the parameters for region Q 
theta_min = 0
theta_max = 2 * np.pi  # Theta ranges from 0 to 2*pi
omega_min = -100
omega_max = 100  # Omega ranges from -100 to 100


# Draw T initial values 
theta_values = np.random.uniform(low=theta_min, high=theta_max, size=T)
#omega_values = np.random.uniform(low=omega_min, high=omega_max, size=T)
omega_values =np.zeros(N)
initial_values = list(zip(theta_values, omega_values))
first_entries = [val[0] for val in initial_values]
second_entries = [val[1] for val in initial_values]


#initial condition inputs for Kuramoto function
phase_ensembles =[] 
freq_ensembles = []
initial_values_for_kuramoto =[]

for pos in range(N):
    ensembles = create_ensembles(pos, first_entries, second_entries, T, θ_s, ω_s)
    phase_group = ensembles[0]
    freq_group = ensembles[1]
    phase_ensembles.extend(phase_group) 
    freq_ensembles.extend(freq_group)

for i in range(len(phase_ensembles)):
    initial_values_for_kuramoto.append([phase_ensembles[i], freq_ensembles[i]])



#Time iterations
t_max = 10000
t_min = 0
t = np.arange(t_min,t_max,1)

#Create Network
G=nx.complete_graph(N)
K_ij=nx.to_numpy_array(G)
#nx.draw(G)

In [152]:
#Simulate Kuramoto for all initial values

results=[]
for i in range(len(phase_ensembles)):
    results.append(simulate_kuramoto(kuramoto_function, initial_values_for_kuramoto[i][0], initial_values_for_kuramoto[i][1], t))

In [154]:
results[0]

(array([[1.30151874, 1.57079633, 1.57079633],
        [1.47203029, 1.48554055, 1.48554055],
        [1.48059248, 1.48125945, 1.48125945],
        ...,
        [1.48103713, 1.48103713, 1.48103713],
        [1.48103713, 1.48103713, 1.48103713],
        [1.48103713, 1.48103713, 1.48103713]]),
 array([ 8.8817842e-16, -4.4408921e-16, -4.4408921e-16]))